```@meta
CurrentModule = ConformalPrediction
```

In [ ]:
#| echo: false
include("$(pwd())/docs/setup_docs.jl")
eval(setup_docs)

# How to Conformalize a Language Model

## Data

In [ ]:
# Get labels:
df_labels = CSV.read("dev/artifacts/data/banking77/labels.csv", DataFrame, drop=[1])
labels = df_labels[:,1]

# Get data:
df_train = CSV.read("dev/artifacts/data/banking77/train.csv", DataFrame, drop=[1])
df_cal = CSV.read("dev/artifacts/data/banking77/calibration.csv", DataFrame, drop=[1])
df_full_train = vcat(df_train, df_cal)
train_ratio = round(nrow(df_train)/nrow(df_full_train), digits=2)
df_test = CSV.read("dev/artifacts/data/banking77/test.csv", DataFrame, drop=[1])

# Preprocess data:
queries_train, y_train = collect(df_train.text), categorical(df_train.labels .+ 1)
queries_cal, y_cal = collect(df_cal.text), categorical(df_cal.labels .+ 1)
queries, y = collect(df_full_train.text), categorical(df_full_train.labels .+ 1)
queries_test, y_test = collect(df_test.text), categorical(df_test.labels .+ 1)

## HuggingFace Model

In [ ]:
tkr = hgf"mrm8488/distilroberta-finetuned-banking77:tokenizer"
mod = hgf"mrm8488/distilroberta-finetuned-banking77:ForSequenceClassification"

In [ ]:
query = [
    "What is the base of the exchange rates?",
    "Exchange rates for the US dollar.",
]
a = encode(tkr, query)
b = mod.model(a)
c = mod.cls(b.hidden_state)
d = softmax(c.logit)
[labels[i] for i in Flux.onecold(d)]

## `MLJ` Models

### Full Model

In [ ]:
struct IntentClassifier <: MLJBase.Probabilistic
    tkr::TextEncoders.AbstractTransformerTextEncoder
    mod::HuggingFace.HGFRobertaForSequenceClassification
end

function IntentClassifier(;
    tokenizer::TextEncoders.AbstractTransformerTextEncoder, 
    model::HuggingFace.HGFRobertaForSequenceClassification,
)
    IntentClassifier(tkr, mod)
end

function get_hidden_state(clf::IntentClassifier, query::Union{AbstractString, Vector{<:AbstractString}})
    token = encode(clf.tkr, query)
    hidden_state = clf.mod.model(token).hidden_state
    return hidden_state
end

# This doesn't actually retrain the model, but it retrieves the classifier object
function MLJBase.fit(clf::IntentClassifier, verbosity, X, y)
    cache=nothing
    report=nothing
    fitresult = (clf = clf.mod.cls, labels = levels(y))
    return fitresult, cache, report
end

function MLJBase.predict(clf::IntentClassifier, fitresult, Xnew)
    output = fitresult.clf(get_hidden_state(clf,Xnew))
    p̂ = UnivariateFinite(fitresult.labels,softmax(output.logit)',pool=missing)
    return p̂
end

MLJBase.target_scitype(clf::IntentClassifier) = AbstractVector{<:Finite}

MLJBase.predict_mode(clf::IntentClassifier, fitresult, Xnew) = mode.(MLJBase.predict(clf, fitresult, Xnew))

In [ ]:
clf = IntentClassifier(tkr, mod)
top_n = 1000
fitresult, _, _ = fit(clf, 1, nothing, y_test[1:top_n])
@time ŷ = predict(clf, fitresult, queries_test[1:top_n]);

### Omniscent Model

In [ ]:
struct OmniscentClassifier <: MLJBase.Probabilistic end

# This doesn't actually retrain the model, but it retrieves the classifier object
function MLJBase.fit(clf::OmniscentClassifier, verbosity, X, y)
    cache=nothing
    report=nothing
    fitresult=y
    return fitresult, cache, report
end

function MLJBase.predict(clf::OmniscentClassifier, fitresult, Xnew)
    p̂ = UnivariateFinite(fitresult.labels,fitresult,pool=missing)
    return p̂
end

MLJBase.target_scitype(clf::OmniscentClassifier) = AbstractVector{<:Finite}

MLJBase.predict_mode(clf::OmniscentClassifier, fitresult, Xnew) = mode.(MLJBase.predict(clf, fitresult, Xnew))

## Conformal Prediction

In [ ]:
cov = 0.95

### Simple Inductive Conformal Prediction

In [ ]:
conf_model = conformal_model(clf; coverage=cov, method=:simple_inductive, train_ratio=train_ratio)
mach = machine(conf_model, queries, y)
@time fit!(mach)
ŷ = predict(mach, queries_test[1:10])

### Adaptive Inductive Conformal Prediction

In [ ]:
conf_model = conformal_model(clf; coverage=cov, method=:adaptive_inductive, train_ratio=train_ratio)
mach = machine(conf_model, queries, y)
@time fit!(mach)
ŷ = predict(mach, queries_test[1:10])